# Project 2 - Mechanics with SymPy

![SymPy logo](https://www.sympy.org/static/images/logo.png)

**SymPy** is a Python library for symbolic mathematics. It aims to become a full-featured computer algebra system (CAS) while keeping the code as simple as possible in order to be comprehensible and easily extensible. It is available under a permissive, commercial-friendly open source license (BSD 3-Clause).

Some of its capabilities are:

- Arithmetic, simplification, expansion, substitution
- Limits, differentiation, integration, Taylor series
- Equation solving
- Matrix algebra
- Physics

SymPy is one of the most active Python projects in terms of contributors.

- **Documentation**: https://docs.sympy.org/latest/index.html
- **Source code**: https://github.com/sympy/sympy/

## Basics of symbolic computation

(_From the_ [_SymPy tutorial_](https://docs.sympy.org/latest/tutorial/intro.html))

Symbolic computation deals with the computation of mathematical objects symbolically. This means that the mathematical objects are represented exactly, not approximately, and mathematical expressions with unevaluated variables are left in symbolic form.

Let’s take an example. Say we wanted to use the built-in Python functions to compute square roots. We might do something like this:

In [ ]:
import math
math.sqrt(9)

9 is a perfect square, so we got the exact answer, 3. But suppose we computed the square root of a number that isn’t a perfect square:

In [ ]:
math.sqrt(8)

But suppose we want to go further. Recall that $\sqrt{8} = \sqrt{4 \cdot 2} = 2 \sqrt{2}$. We would have a hard time deducing this from the above result. This is where symbolic computation comes in. With a symbolic computation system like SymPy, square roots of numbers that are not perfect squares are left unevaluated by default:

In [ ]:
import sympy as sp
sp.sqrt(3)

Furthermore—and this is where we start to see the real power of symbolic computation—symbolic results can be symbolically simplified:

In [ ]:
sp.sqrt(8)

### A more interesting example

Symbolic computation systems (which by the way, are also often called computer algebra systems, or just CASs) such as SymPy are capable of computing symbolic expressions with variables.

Let us define a symbolic expression, representing the mathematical expression $x+2y$:

In [ ]:
x, y = sp.symbols("x y")
expr = x + 2 * y
expr

Note that we wrote `x + 2*y` just as we would if `x` and `y` were ordinary Python variables. But in this case, instead of evaluating to something, the expression remains as just `x + 2*y`. Now let us play around with it:

In [ ]:
expr + 1

In [ ]:
expr - x

In [ ]:
x * expr

Here, we might have expected $x(x+2y)$ to transform into $x^2+2xy$, but instead we see that the expression was left alone. This is a common theme in SymPy. Aside from obvious simplifications like $x−x=0$ and $\sqrt{8} = 2\sqrt{2}$, most simplifications are not performed automatically. This is because we might prefer the factored form $x(x+2y)$, or we might prefer the expanded form $x^2+2xy$. Both forms are useful in different circumstances. In SymPy, there are functions to go from one form to the other:

In [ ]:
expanded_expr = sp.expand(x * expr)
expanded_expr

In [ ]:
sp.factor(expanded_expr)

There are many more things that can be done: differentiating and integrating expressions with respect to variables, computing limits, solving algebraic and differential equations... Check out the [SymPy tutorial](https://docs.sympy.org/latest/tutorial/index.html) for more!

## Mechanics

### Reference frames

To study Mechanics and Kinematics using SymPy, we need to start by defining our own reference frames. Then we will use them to express our geometrical relations and transform vectors between them.

In [ ]:
import sympy as sp
sp.init_printing()

from sympy.physics.mechanics import ReferenceFrame

# We define our own class so our unit vectors are (i, j, k)
class IJKReferenceFrame(ReferenceFrame):
    def __init__(self, name):
        super().__init__(name, latexs=['\mathbf{%s}_{%s}' % (idx, name) for idx in ("i", "j", "k")])
        self.i = self.x
        self.j = self.y
        self.k = self.z

We define our own reference frame like this:

In [ ]:
A = IJKReferenceFrame("1")
A.i

And to write vectors in this frame we **multiply each component by the corresponding unit vector**:

In [ ]:
2 * A.i - 1 * A.j

### Vector algebra

Our vectors work like symbols, and we can perform dot and cross products with them:

In [ ]:
R, V = sp.symbols("R, V", positive=True)
r1 = R * (A.i + A.j + A.k)
v1 = V * (A.i - 2 * A.k)

In [ ]:
r1

In [ ]:
v1

In [ ]:
r1.dot(v1)
r1 & v1  # Alternative to .dot

In [ ]:
r1.cross(v1)
r1 ^ v1  # Alternative to .cross

We can compute their `.magnitude()` and `.normalize()` them:

In [ ]:
(r1 ^ v1).magnitude()

In [ ]:
(r1 ^ v1).normalize()

### Exercise

Using directly the Transport Theorem, Basic Kinematic Equation or formula for time derivative in a rotating frame:

$$\left(\frac{\operatorname{d}\!\mathbf{a}}{\operatorname{d}\!t}\right)_1 = \left(\frac{\operatorname{d}\!\mathbf{a}}{\operatorname{d}\!t}\right)_0 + \mathbf{\omega}_{01}\! \times \mathbf{a}$$

Compute the derivative of this vector:

$$
\mathbf{a} = R \mathbf{i}_0
$$

where $A_0$ is a reference frame that rotates with respect to the inertial frame with angular velocity $\mathbf{\omega}_{01}=\Omega \mathbf{k}_0$. **What's the magnitude of the derivative?**

In [ ]:
# Write your solution here!
# Remember to insert extra cells if you need it


### Dynamic symbols

There is a straightforward way of creating symbols that have a time dependence using `dynamicsymbols`:

In [ ]:
from sympy.physics.mechanics import dynamicsymbols

In [ ]:
α = dynamicsymbols("α")
α

And compute its derivative using `.diff()`:

In [ ]:
α.diff()

### Relative motion

SymPy provides powerful methods to compute rotation matrices between two reference frames. For that we need to specify their relative orientation using `.orient`, and then we can recover the direct cosine matrix using `.dcm`:

In [ ]:
A0 = IJKReferenceFrame("0")
A1 = IJKReferenceFrame("1")

In [ ]:
ϕ = dynamicsymbols("ϕ")
A1.orient(A0, "Axis", [ϕ, A0.z])  # Rotation ϕ around A1.z axis
A1.dcm(A0)

And we can obtain its angular velocity using `.ang_vel_in`:

In [ ]:
A1.ang_vel_in(A0)

Using value `Axis` we specified the rotation around an axis. Other methods are:

* `Body`: three Euler angles.
* `Space`: like `Body`, but the rotations are applied in the reverse order.
* `Quaternion`: rotating around a unit vector $\mathbf{\lambda}$ a quantity $\theta$.

To express a vector in a different reference frame, we can use `.express` or `.to_matrix`:

In [ ]:
A0.i.express(A1)

In [ ]:
A0.i.to_matrix(A1)

### Time derivative in rotating frame

Rather than applying the Basic Kinematic Equation manually as we did above, SymPy can do that automatically:

In [ ]:
v1 = A1.x
v1

In [ ]:
dv1 = v1.diff(sp.symbols("t"), A0)
dv1

In [ ]:
dv1.to_matrix(A1)

In [ ]:
(dv1 & A1.j).simplify()

### Exercise

![Reference frames of a rotor blade](img/rotor-blade.jpg)

(Cuerva et al. "Teoría de los Helicópteros")

1. Obtain the direct cosine matrix of the blade $B$ with respect to $A1$.
2. Obtain the angular velocity of $B$ with respect to $A1$, expressed in $A$.

In [ ]:
# Write your solution here!
# Remember to insert extra cells if you need it


### Points and velocities

The last step in our kinematics study is the possibility to define points in solids and compute their velocity field. For this, SymPy provides the `Point` class:

In [ ]:
from sympy.physics.mechanics import Point

In [ ]:
O = Point("O")

To set a point $O$ as the origin of $A$, we will define its velocity to be zero with `.set_vel`:

In [ ]:
A = IJKReferenceFrame("A")
A1 = IJKReferenceFrame("A1")
ψ = dynamicsymbols("ψ")
A1.orient(A, "Axis", [ψ, A.z])

In [ ]:
O.set_vel(A, 0)

Next, to define new points, we use the method `.locate_new`:

In [ ]:
e_b = sp.symbols("e_b")
E_b = O.locatenew("E_b", e_b * A1.x)

To obtain the position vector of one point with respect to the other, we use `.pos_from`:

In [ ]:
E_b.pos_from(O)

And finally, the **velocity field of a rigid body** is obtained using `.v2pt_theory`:

$$v^P_A = v^O_A + \omega_{A_1 A} \times \mathbf{OP}$$

This method belongs to _the point we want to study_ and receives three parameters:

* `O`: point with known velocity in $A$
* `A`: reference frame where we want to compute the velocity
* `A1`: reference frame where both points are fixed

Therefore, to compute the velocity of the point we just created:

In [ ]:
E_b.v2pt_theory(O, A, A1)

### The no-slip disc

![No-slip disc](img/no-slip-disc.png)

(Notes from Óscar López Rebollal, Polytechnic University of Madrid)

**Compute the velocity and acceleration of point $P$**.

In [ ]:
A1 = IJKReferenceFrame("1")
A0 = IJKReferenceFrame("0")
A2 = IJKReferenceFrame("2")

In [ ]:
ξ, θ = dynamicsymbols("ξ, θ")
ξ, θ

In [ ]:
A0.orient(A1, "Axis", [0, A1.k])  # A0 does not rotate with respect to A1
A2.orient(A0, "Axis", [θ, A0.k])

In [ ]:
A2.dcm(A1)

In [ ]:
C = Point("C")  # Center of the disc
C.set_vel(A1, ξ.diff() * A1.i)

In [ ]:
# We locate P, fixed point on the disc, with respect to C, in A2 (rotating with the disc)
R = sp.symbols("R", positive=True)
P = C.locatenew("P", -R * A2.j)
P.pos_from(C)

In [ ]:
# Velocity of P in A1
# With this function call, we express that C and P are fixed in A2
P.v2pt_theory(C, A1, A2)

In [ ]:
P.v2pt_theory(C, A1, A2).express(A0)

**Mission accomplished 🚀**

## References

- Vector documentation (kinematics) https://docs.sympy.org/latest/modules/physics/vector/index.html
- Rigid bodies and more https://docs.sympy.org/latest/modules/physics/mechanics/masses.html
- Dynamics, linearization... https://docs.sympy.org/latest/modules/physics/mechanics/

(_This material is based on [Curso AeroPython by Juan Luis Cano Rodriguez and Alejandro Sáez Mollejo](https://github.com/AeroPython/Curso_AeroPython) (CC-BY)_)